In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")

In [3]:
# imports
import warnings
warnings.filterwarnings("ignore")

import time
import random
import openai
import json
from typing import Union
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from mega.data.data_utils import choose_few_shot_examples
from mega.prompting.instructions import INSTRUCTIONS
from mega.prompting.prompting_utils import load_prompt_template
from mega.utils.env_utils import load_openai_env_variables
from mega.models.completion_models import get_model_pred, gpt3x_completion
from mega.prompting.prompting_utils import construct_prompt, construct_cmsentiment_prompt
from tqdm import tqdm

In [4]:
# Make sure that {env_name}.env file is present in the envs/ directory
env_name = "melange"
load_env(env_name=env_name)

In [5]:
openai.api_base

'https://gpttesting1.openai.azure.com/'

In [6]:
random.seed(42)
np.random.seed(42)

Get data

In [7]:
# Read train.txt and test.txt files
with open('data/code-mixed-datasets/sentiment/train.txt', 'r') as f:
    train_lines = f.readlines()
with open('data/code-mixed-datasets/sentiment/validation.txt', 'r') as f:
    test_lines = f.readlines()

# Create dictionaries with "text" and "label" keys and corresponding values from train.txt and test.txt
train_dict = {"text": [], "label": []}
for line in train_lines:
    text, label = line.strip().split('\t')
    train_dict["text"].append(text)
    train_dict["label"].append(label)

test_dict = {"text": [], "label": []}
for line in test_lines:
    text, label = line.strip().split('\t')
    test_dict["text"].append(text)
    test_dict["label"].append(label)

# # Create Dataset objects
train_dataset = Dataset.from_dict(train_dict)
test_dataset = Dataset.from_dict(test_dict)

# Create DatasetDict with "train" and "test" keys
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1851
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 231
    })
})

In [43]:
model = "gpt-35-turbo"
prompt_name = "following positive negative neutral"
few_shot_k = 8

In [25]:
train_dataset[:5]

{'text': ['campeon really hit me in the heart ! ! !',
  'hahah todo es bien pues , que estas haciendo que no te has dormido lol',
  'y nada de fat puro muscle',
  'stupid .. i think u do sometimes ! ! : o ya se por quien tmb ... lol',
  'yeaaa people will always bitch and have shit to say ! i was tld my damas looked better then me'],
 'label': ['positive', 'positive', 'neutral', 'negative', 'negative']}

In [26]:
template = """Does the following sentence have a positive, negative or neutral sentiment? {text}"""

verbalizer = { "positive": "positive", "negative": "negative", "neutral": "neutral"}


In [27]:
# Loading instruction for the task
instruction = INSTRUCTIONS["gluecos_sentiment"]
print(instruction)

You are an NLP assistant whose purpose is to solve Sentiment Analysis problems. Sentiment Analysis is the task of determining whether the sentiment, opinion or emotion expressed in a textual data is: positive, negative, or neutral. Answer as concisely as possible in the same format as the examples below:


In [44]:
# Getting few-shot examples
random.seed(42)
np.random.seed(42)
train_examples = choose_few_shot_examples(
        train_dataset, few_shot_k, selection_criteria="random-stratified")

In [45]:
train_examples

[{'text': 'olvidalo tonto .. u got jokes tonight ! ! lol and maybe u should give me a job boss lmao jk im glad siguele por buen camino :)',
  'label': 'positive'},
 {'text': 'she asked me " mija me puedes limpiar la cocina esque no a dormido " of course ma there is no need to even ask',
  'label': 'neutral'},
 {'text': 'ahhh vergas ! ! ! aparesco chino tu you ? ?', 'label': 'neutral'},
 {'text': 'do me a paro yeah ? get me a remix gram ? i got the diego',
  'label': 'neutral'},
 {'text': 'pay back es lo q veo se usa mas :] y si pagar con la misma moneda .',
  'label': 'negative'},
 {'text': 'guanajuato gano tonight en el carnaval eso mero mi jente (; ! ! !',
  'label': 'positive'},
 {'text': 'no pues casi todas lmao , watch the news lol that always puts me rite to sleep xd',
  'label': 'negative'}]

In [46]:
test_example = test_dataset[0]

prompt, label = construct_cmsentiment_prompt(
    train_examples,
    test_dataset[0],
    train_prompt_template=template,
    test_prompt_template=template,
    chat_prompt=True if model != "DaVinci003" else False,
    instruction=instruction,
    verbalizer=verbalizer
)
prompt

'Does the following sentence have a positive, negative or neutral sentiment? olvidalo tonto .. u got jokes tonight ! ! lol and maybe u should give me a job boss lmao jk im glad siguele por buen camino :)\npositive\nDoes the following sentence have a positive, negative or neutral sentiment? she asked me " mija me puedes limpiar la cocina esque no a dormido " of course ma there is no need to even ask\nneutral\nDoes the following sentence have a positive, negative or neutral sentiment? ahhh vergas ! ! ! aparesco chino tu you ? ?\nneutral\nDoes the following sentence have a positive, negative or neutral sentiment? do me a paro yeah ? get me a remix gram ? i got the diego\nneutral\nDoes the following sentence have a positive, negative or neutral sentiment? pay back es lo q veo se usa mas :] y si pagar con la misma moneda .\nnegative\nDoes the following sentence have a positive, negative or neutral sentiment? guanajuato gano tonight en el carnaval eso mero mi jente (; ! ! !\npositive\nDoes t

In [47]:
prediction = gpt3x_completion(
    prompt,
    model,
    temperature=0,
    max_tokens=10
)
match = float(prediction.startswith(label))
print(f"Prediction: {prediction}")
print(f"Label: {label}")
print(f"Match: {match}")

Prediction: positive
Label: neutral
Match: 0.0


In [48]:
matches = []
preds = []
labels = []
pbar = tqdm(test_dataset)
for test_example in pbar:
    prompt, label = construct_cmsentiment_prompt(
        train_examples,
        test_example,
        train_prompt_template=template,
        test_prompt_template=template,
        chat_prompt=True if model != "DaVinci003" else False,
        instruction=instruction,
        verbalizer=verbalizer
    )
    prediction = gpt3x_completion(
        prompt,
        model,
        temperature=0,
        max_tokens=10
    )
    time.sleep(1/2)
    match = float(prediction.startswith(label))
    preds.append(prediction)
    labels.append(label)
    matches.append(match)
    running_acc = np.mean(matches)
    pbar.set_description(f"Accuracy: {running_acc}")

print(f"Accuracy: {np.mean(matches)}")

Accuracy: 0.6883116883116883: 100%|█████████████████████████████████████████████████████████████| 231/231 [05:47<00:00,  1.50s/it]

Accuracy: 0.6883116883116883



Bad pipe message: %s [b'\xa6\x14\xea,\xb8.\x02\xe0C', b'+\xbd\x86\x8aY\xacC \xbcS\xc9\x98\x9e$\xfe:\xfcLK\x96\xbd\x95_\x83\x89\x18T\x8b\xbd\xc1\x87\xd8\xc0&\n\xdb\xe1\x91Y\xed\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00']
Bad pipe message: %s [b'\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07']
Bad pipe message: %s [b'\x08\t\x08\n\x08\x0b\x08']
Bad pipe message: %s [b'\x05\x08\x06']
Bad pipe message: %s [b'\x05\x01\x06', b'']
Bad pipe message: %s [b"et\x8fQ4J\xb6\x00\xfd\x8e\x17?\xa5\x16\x93\x88\xa7b\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\